<h2>This script is for internal developer testing.</h2>

In [1]:
import wfdb

In [ ]:
# Testing rdsamp multiple samples per frame (and fmt 12)
sig, fields=wfdb.srdsamp('sampledata/03700181')
print(sig[:20,:])
print("\n\n", sig.shape, "\n\n")
print(fields)

print("\n\n\n")
print(sig[-20:,:])

In [ ]:
# Testing rdsamp format 80
sig, fields=wfdb.srdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
sig, fields=wfdb.srdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import rdsamp

# Return list of arrays
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=3200, sampto=20000, stacksegments=0)
#print("len(sig): ", len(sig))
#print(len(fields))

print(sig)
print("\n\n")
print(sig[len(sig)-1])
print("\n\n")
print(len(sig[len(sig)-1]))
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-segment variable layout record - stacksegments
import numpy as np
from wfdb import rdsamp


# Return single numpy array
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=1, sampto=10000, channels=[1, 2], physical=1)

print(sig)
print(sig.shape)
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
x=[1,2,3]
y=[x]*3
y[1]

In [ ]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)
import numpy as np
from wfdb import rdsamp

sig, fields=rdsamp('sampledata/s0010_re', channels=[14, 0, 5, 10, 11], sampfrom=100, sampto=20000)
from wfdb import plotwfdb
plotwfdb.plotsigs(sig, fields)
print("\n\n\n", sig, "\n\n\n")
print(fields)
print(fields["signame"])

In [ ]:
# Testing rdsamp multi-segment record with sampfrom, sampto, and channel selection
import numpy as np
from wfdb import rdsamp

# Return list of arrays
#sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=10, sampto=10250, channels=[2,3], stacksegments=0)
# rdsamp -r s00001-2896-10-10-00-31 -s 2 3 -t 82 -P
#print("\n\nOutput of rdsamp:")
#print("sig: ", sig)
#print("fields: ", fields)


# Return single array
sig, fields=rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=10, sampto=10250, channels=[1,0,2],stacksegments=1)

print(sig)
print(fields[2])

<h2>Decide comprehensive tests before adding to travis - signals</h2>
Target files created with wfdb software package's <code>rdsamp</code>

In [ ]:
import numpy as np
from wfdb import rdsamp

# Test 1 - Format 212/Entire signal/Physical 
# Target file created with: rdsamp -r sampledata/100 -P | cut -f 2- > target1
sig, fields=rdsamp('sampledata/100') 

# Test 2 - Format 212/Selected Duration/Selected Channel/Digital
# Target file created with: rdsamp -r sampledata/100 -f 1 -t 30 -s 1 | cut -f 2- > target2
sig, fields=rdsamp('sampledata/100', sampfrom=360, sampto=10800, channels=[1], physical=0)  

# Test 3 - Format 16/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/test01_00s | cut -f 2- > target3
sig, fields=rdsamp('sampledata/test01_00s', physical=0)

# Test 4 - Format 16 with byte offset/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/a103l -f 50 -t 160 -s 2 0 -P | cut -f 2- > target4
sig, fields=rdsamp('sampledata/a103l', sampfrom=12500, sampto=40000, channels=[2, 0])

# Test 5 - Format 16 with byte offset/Selected Duration/Selected Channels/Digital
# Target file created with: rdsamp -r sampledata/a103l -f 80 -s 0 1 | cut -f 2- > target5
sig, fields=rdsamp('sampledata/a103l', sampfrom=20000, physical=0, channels=[0, 1])

# Test 6 - Format 80/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/3000003_0003 -f 1 -t 8 -s 1 -P | cut -f 2- > target6
sig, fields=rdsamp('sampledata/3000003_0003', sampfrom=125, sampto=1000, channels=[1])

# Test 7 - Multi-dat/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/s0010_re | cut -f 2- > target7
sig, fields=rdsamp('sampledata/s0010_re', physical=0)

# Test 8 - Multi-dat/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/s0010_re -f 5 -t 38 -P -s 13 0 4 8 3 | cut -f 2- > target8
sig, fields=rdsamp('sampledata/s0010_re', sampfrom=5000, sampto=38000, channels=[13, 0, 4, 8, 3])

# Test 9 - Format 12 multi-samples/frame and skew/Entire Signal/Digital
# Target file created with: rdsamp -r sampledata/03700181 | cut -f 2- > target9
sig, fields=rdsamp('sampledata/03700181', physical=0)

# Test 10 - Format 12 multi-samples/frame and skew/Selected Duration/Selected channel/Physical
# Target file created with: rdsamp -r sampledata/03700181 -f 8 -t 128 -P | cut -f 2- > target10
sig, fields=rdsamp('sampledata/03700181', channels=[0, 2], sampfrom=1000, sampto=16000)

print('yay')

<h2>Decide comprehensive tests before adding to travis - annotations</h2>
Target files created with wfdb software package's <code>rdann</code>

In [ ]:
import numpy as np
from wfdb import rdann

# Test 1 - 
# Target file created with: rdann -r 100 -a atr > anntarget1
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/100', 'atr')


# Test 2
# Target file created with: rdann -r chb01_03.edf -a seizures > anntarget2
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/chb01_03', 'seizures')


In [ ]:
from wfdb import rdann
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/100', 'fake2', sampfrom=17)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)


a=aux[3]
print(a)
print(len(a))
a=aux[5]
print(a)
print(len(a))

In [ ]:
from wfdb import rdann
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/chb01_03.edf', 'seizures', sampfrom=766977, sampto=777326)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
from wfdb import rdann
annsamp, anntype, num, subtype, chan, aux, annfs=rdann('sampledata/100', 'fake3', anndisp=1)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
# For writing test annotation files based on first 3 annotations of 12726.anI

f=open('tmp.fake', 'wb')

indata= np.array([0, 236,   1,   0, 200,  84,   0,  88, 255, 251,  21, 252,  73, 110, 105, 116,
                  105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
                  112,   0,   0, 236,   0,   0,  67,  50,   0,  88,  21, 252,  67, 111, 110,  99,
108, 117, 100, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
112,   0,   0, 236,   0,   0, 114, 183,   0,  88,  23, 252,  73, 110, 105, 116,
105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 100,
111, 119, 110,   0, 0 ,0 ]).astype('u1')


print(indata)

indata.tofile(f)


f.close()


In [ ]:
# Write a single sample format 212 wfdb record to read
import numpy as np

f=open('tmp.dat', 'wb')
indata= np.array([100, 0]).astype('u1')
indata.tofile(f)
f.close()

from wfdb import rdsamp
sig, fields=rdsamp('tmp')
print(sig)


In [ ]:
# Write a 1 channel fmt 12 wfdb record and test reading specific samples
import numpy as np

f=open('single12.dat', 'wb')
indata= np.array([20, 0, 30, 40, 0, 50 ]).astype('u1')
indata.tofile(f)
f.close()


from wfdb import readsignal
sig, fields=readsignal.rdsamp('single12', physical=0, sampfrom=3)
print(sig)

In [ ]:
# Write a 1 channel fmt 310 wfdb record and test reading specific samples
import numpy as np
from wfdb import readsignal

f=open('single310.dat', 'wb')
indata= np.array([20, 0, 0, 0, 20, 0, 0, 0]).astype('u1')
indata.tofile(f)
f.close()



sig, fields=readsignal.rdsamp('single310', physical=0, sampfrom=5)
print(sig)

In [ ]:
# Testing format 310 record. Written using rdsamp -r sampledata/100 | cut -f 2- | wrsamp -o test310 -O 310 
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/test310', physical=0, sampto=1)
print("sig final output:\n", sig)
#print(sig.shape)



# WFDB did not write the last sample???? 

In [ ]:
# Testing format 311 record. Written using rdsamp -r sampledata/100 | cut -f 2- | wrsamp -o test311 -O 311
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/test311', physical=0)
print("sig final output:\n", sig)
#print(sig.shape)



# WFDB did not write the last sample???? 

In [ ]:
from wfdb import rdsamp
sig, fields=rdsamp('sampledata/s0010_re', sampfrom=5000, sampto=38000, channels=[13, 0, 4, 8, 3])
sig=sig[:-4,:]


targetsig=np.genfromtext('tests/targetoutputdata/target9')
np.array_equal(sig, targetsig)

In [ ]:
# Trying to fix byte pointer for reading skewed signals. 

from wfdb import readsignal
import numpy as np
#sig, fields=readsignal.rdsamp('sampledata/03700181', channels=[0,2], sampfrom=1000, sampto=16000)
#sig=np.round(sig, decimals=8)
#targetsig=np.genfromtxt('tests/targetoutputdata/target10')

sig, fields=readsignal.rdsamp('sampledata/03700181', sampfrom=1000, sampto=16000, physical=0) # fs=125. -f 8 -t 128

print("\nsig: ", sig)
print('\n\n')
print("End of sig: ", sig[-10:,:])


# nbytesread = 15000 frames * 6 samples/frame * 1.5 bytes/sample = 135000

# Try to octal dump the right section of the file. After 16000 frames x (6 samples/frame)  = 96000 samples * (1.5 bytes per sample)
# = 144000 bytes skip. 



# od -t u1 03700181.dat -j 144000 -N 30 

# Result: 
# 236 255 236   2 240 236 163  27  61 236 255 236 236  15   2 147
#  27  40 236 255 236 236 255 236 130  27  19   2 240 236




In [ ]:
from wfdb import readsignal
import numpy as np
sig, fields=readsignal.rdsamp('sampledata/matched/s25047/s25047-2704-05-04-10-44')
sig=np.round(sig, decimals=8)
targetsig=np.genfromtxt('tests/targetoutputdata/target11')

print(sig)
print(targetsig)

In [ ]:
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/matched/s25047/s25047-2704-05-04-10-44', physical=0)

print(sig)

In [ ]:
# Layout header gains: 86, 86
# Specific record gains: 81, 60

# Digital samples: 4, 2 (second last row) 


# Physical according to layout: 0.0465, 0.02325
# Physical according to specific record: 0.0493, 0.0333


# Physical according to WFDB software rdsamp: 0.0465   0.02325


In [ ]:
# Current logic with omission: 

# First, split the recordname into 2. Assume base 

pbsubdir, baserecordname = os.path.split(recordname) # Make assumption that the base is a physiobank 


downloaddir=cachedir+pbsubdir # This could be wrong, ie: /usr/local/database/testfolder


if downloaddir==current directory: # This could therefore be wrongly assumed
    
    check for all needed files in download directory. 
    Download anything missing.
    
    
else: # downloaddir != current directory         This could be wrongly assumed. If so, all is fucked. 
    
    check for files in current directory. 
    if any missing: 
        getpbrecords(recordname, downloaddir)
    
    
    
    

In [ ]:

# Maybe this should be multiple functions....





# Desired logic:


# Need to distinguish between: recordname=testfolder/100 and recordname=mitdb/100 




# First, split the recordname into 2:

basedir, baserecordname = os.path.split(recordname) # basedir could be a physiobank database or a directory


# If basedir is empty, then just return immediately the same recordname because [] means not targeting a physiobank 
# database. Nothing to download so no point checking. Also path means current path. 



# Need to hold off calculating what download directory means, or put a flag specifying whether we can guarantee it
# as one of the input parameters. 



# If basedir is not empty, check whether the directory exists (from the current working dir:

if os.path.isdir(basedir):# If the directory does exist, go into it and check for all files. If any are missing, then 
    # they are fucked anyway because they didn't specify a physiobank directory. 
    # So therefore no point checking for files and just return immediately. 
    
    
    
    # Wait... so what if they say: mitdb/100 and they happen to have a mitdb directory from where they currently are? 
    # This would probably be quite common too since they might start python in the dbcachedir. 
    
    # So we have to 
    
    
    
else: # If the directory does not exist. Then assume it is a physiobank directory. Go call  
    
    # check the current
    
    

In [ ]:
import os
import wfdb

wfdb.rdsamp.dlrecordfiles('macecgdb/test01_00s', '/home/cx1111/Downloads/wfdbrecords/prcp')

In [ ]:
# Make sure it works for 8 scenarios of input arguments and cases:

# 1. recordname=mitdb/100 
# 2. recordname=testfolder/100

# a. filedirectory=dbcachedir
# b. filedirectory!=dbcachedir

# i. mitdb or testfolder exists
# ii. mitdb or testfolder does not exist. 

        
# 1ai
# 1aii
# 1bi
# 1bii
# 2ai
# 2aii
# 2bi
# 2bii
        
        

In [ ]:
import wfdb
sig, fields=wfdb.rdsamp('mimic2wdb/matched/s00001/s00001-2896-10-10-00-31')

print(sig)

In [ ]:
import wfdb
sig, fields=wfdb.rdsamp('macecgdb/test01_00s', pbdl=1, dldir='/home/cx1111/Downloads/wfdbrecords/macecgdb')

print(sig)

In [ ]:
import wfdb
sig, fields=wfdb.rdsamp('mitdb/101', pbdl=1, dldir='/home/cx1111/Downloads/wfdbrecords/mitdb', keepfiles=1)

print(sig)

In [ ]:
import wfdb

wfdb.rdsamp('macecgdb/test01_00s', pbdl=1)


In [ ]:
l = ['hi','ack', 'hi','bye','bye','hi','hi']

In [ ]:
# Returns the unique elements in a list in the order that they appear. 
# Also returns the indices of the original list that correspond to each output element. 
# Used for listing dat files and fmts to write. 
# Belongs to module
def orderedsetlist(fulllist):
    uniquelist = []
    listindices = []
    for i in fulllist:
        # Found a new item
        if i not in uniquelist:
            uniquelist.append(i)
            listindices.append([a for a, e in enumerate(fulllist) if e == i])

    return uniquelist, listindices

# Returns elements in a list without consecutive repeated values. Used to check dat files to write. 
def orderednoconseclist(fulllist):
    noconseclist = [fulllist[0]]
    if len(fulllist) == 1:
        return noconseclist
    for i in fulllist:
        if i!= noconseclist[-1]:
            noconseclist.append(i)
    return noconseclist
        

In [ ]:
print(orderedsetlist(mylist))
print(orderednoconseclist(mylist))

In [ ]:
mylist = ['a','a','a','b','b','c','d','d']

In [ ]:
targetoutput = [(0,2), (3,4), (5,5), (6,7)]

In [ ]:
for count, elem in enumerate(mylist):
    print(count, elem)

In [ ]:
help(enumerate)

In [ ]:
import re

a = re.match('\d+', 'hello')

print(a)
#a.group()

In [ ]:
acceptedstring = re.match('[\w\d]+\.[\w\d]+','100adat')
acceptedstring

In [ ]:
import numpy as np
def calc_checksum(signals):
    return np.sum(signals, 0) % 65536

z = np.array([[1,2,3,4,5], [7,7,7,7,7]])

list(calc_checksum(z))

list(z[0,:])

In [ ]:
class bread():
    
    def __init__(self):
        self.type = 'white'
        self.weight = 2
        self.volume = bread.mathsfunction(10,10)
        
    def mathsfunction(a,b):
        return a*b
    


In [ ]:
bread.mathsfunction(10,10)

In [ ]:
try:
    parsedatestring('34/09/1992')
except SystemExit:
    print('nope')

In [ ]:
from wfdb import _rdsamp
sig, fields = _rdsamp.rdsamp('/home/chen/Projects/wfdb-python/sampledata/100')
print(sig)
print(fields)

In [ ]:
import wfdb

record = wfdb.rdsamp('/home/chen/Projects/wfdb-python/sampledata/100', physical = 1)

# 1. Initiate the WFDBrecord object with essential information. 
#record1 = records.WFDBrecord(recordname = 'record1', p_signals = sig, fs = 360, units = ['mV','mV'], signame = ['I','V'])
# Alternatively you can also add in the fmt
record1 = wfdb.WFDBrecord(recordname = 'record1', p_signals = record.p_signals, fs = 360, fmt = ['16','16'], units = ['mV','mV'], signame = ['I','V'])

# 2. Compute optimal fields to store the digital signal, carry out adc, and set the fields.
record1.set_d_features(do_adc = 1)
# 3. Set default values of any missing field dependencies
record1.setdefaults()
# 4. Write the record files - header and associated dat
record1.wrsamp()

In [ ]:
print(record.d_signals)
print(record.__dict__)
print(record.p_signals)
print(record1.p_signals)

In [ ]:
import wfdb
a = wfdb.rdsamp('/home/chen/Projects/wfdb-python/sampledata/100')

In [ ]:
a=np.zeros([4,3])

In [ ]:
hasattr(a, 'shape')

In [ ]:
seglen = [100,200,300,200]
startsamps = [0] + list(np.cumsum(seglen)[0:-1])
endsamps = list(np.cumsum(seglen))

print(startsamps)
print(endsamps)

In [ ]:
# Figure out if there are any channels wanted and 
# the output channels they are to be stored in

fields = {}
fields['signame'] = ['I','II','IV','V']
fields['nsig'] = 4




inchannels = []
outchannels = []
for s in fields['signame']:
    if s in seg.signame:
        inchannels.append(seg.signame.index(s))
        outchannels.append(fields['signame'].index(s))


for ch in range(0, fields['nsig']):
    if ch not in outchannels:
        p_signals[:,ch] = np.nan
    else:
        p_signals[:,ch] = seg.p_signals[:, inchannels[outchannels.index(ch)]]

In [ ]:
import importlib
import wfdb
importlib.reload(wfdb)
#a = wfdb.rdsamp('/home/chen/Projects/wfdb-python/sampledata/matched/s00001/s00001-2896-10-10-00-31')
a = wfdb.rdsamp('/home/cx1111/Projects/wfdb-python/sampledata/matched/s00001/s00001-2896-10-10-00-31')


In [ ]:
a.__dict__

In [ ]:
import numpy as np
def givex():
    x = np.array([[1,2,3],[1,2,3],[1,2,3]])
    return x

givex()[:,[1,1,0,1]]


In [4]:
import numpy as np
        
np.cumsum([1,2,3,4,5])

array([ 1,  3,  6, 10, 15])

In [13]:
a = np.diff([1,2,3,4,59,99,33])

a.dtype == 'int64'

a.ndim

1

In [6]:
min([1,2,3])


1

In [12]:
b = np.array(['a','b','c

array(['a', 'b', 'c'], 
      dtype='<U1')

In [14]:
# Mask of 6 msb out of 16: 2**10 + 2**11 + ...

typemask = 2**10 + 2**11 + 2**12 + 2**13 + 2**14 + 2**15

(1317 & typemask) >> 10
(8995 & typemask) >> 10


8

In [33]:
import numpy as np
a = set(np.array([99,0,1,2,3,4,5]))
b = set([1,2,3,4,5,6])

c = set(a) - set(b)
c
#c == set()

{0, 99}

In [36]:
d = {1:1, 2: 'b'}
set(d.values())

{'b', 1}

In [45]:
# Annotation mnemonic symbols for the 'anntype' field as specified in annot.c
# from wfdb software library 10.5.24
annsyms = {
    0: ' ',  # not-QRS (not a getann/putann codedict) */
    1: 'N',  # normal beat */
    2: 'L',  # left bundle branch block beat */
    3: 'R',  # right bundle branch block beat */
    4: 'a',  # aberrated atrial premature beat */
    5: 'V',  # premature ventricular contraction */
    6: 'F',  # fusion of ventricular and normal beat */
    7: 'J',  # nodal (junctional) premature beat */
    8: 'A',  # atrial premature contraction */
    9: 'S',  # premature or ectopic supraventricular beat */
    10: 'E',  # ventricular escape beat */
    11: 'j',  # nodal (junctional) escape beat */
    12: '/',  # paced beat */
    13: 'Q',  # unclassifiable beat */
    14: '~',  # signal quality change */
    #15: '[15]',
    16: '|',  # isolated QRS-like artifact */
    #17: '[17]',
    18: 's',  # ST change */
    19: 'T',  # T-wave change */
    20: '*',  # systole */
    21: 'D',  # diastole */
    22: '"',  # comment annotation */
    23: '=',  # measurement annotation */
    24: 'p',  # P-wave peak */
    25: 'B',  # left or right bundle branch block */
    26: '^',  # non-conducted pacer spike */
    27: 't',  # T-wave peak */
    28: '+',  # rhythm change */
    29: 'u',  # U-wave peak */
    30: '?',  # learning */
    31: '!',  # ventricular flutter wave */
    32: '[',  # start of ventricular flutter/fibrillation */
    33: ']',  # end of ventricular flutter/fibrillation */
    34: 'e',  # atrial escape beat */
    35: 'n',  # supraventricular escape beat */
    36: '@',  # link to external data (aux contains URL) */
    37: 'x',  # non-conducted P-wave (blocked APB) */
    38: 'f',  # fusion of paced and normal beat */
    39: '(',  # waveform onset */
    # 39: 'PQ', # PQ junction (beginning of QRS) */
    40: ')',  # waveform end */
    # 40: 'JPT', # J point (end of QRS) */
    41: 'r',  # R-on-T premature ventricular contraction */
    #42: '[42]',
    #43: '[43]',
    #44: '[44]',
    #45: '[45]',
    #46: '[46]',
    #47: '[47]',
    #48: '[48]',
    #49: '[49]',
}

# Annotation codes for 'anntype' field as specified in ecgcodes.h from
# wfdb software library 10.5.24
anncodes = {
    0: 'NOTQRS',  # not-QRS (not a getann/putann codedict) */
    1: 'NORMAL',  # normal beat */
    2: 'LBBB',  # left bundle branch block beat */
    3: 'RBBB',  # right bundle branch block beat */
    4: 'ABERR',  # aberrated atrial premature beat */
    5: 'PVC',  # premature ventricular contraction */
    6: 'FUSION',  # fusion of ventricular and normal beat */
    7: 'NPC',  # nodal (junctional) premature beat */
    8: 'APC',  # atrial premature contraction */
    9: 'SVPB',  # premature or ectopic supraventricular beat */
    10: 'VESC',  # ventricular escape beat */
    11: 'NESC',  # nodal (junctional) escape beat */
    12: 'PACE',  # paced beat */
    13: 'UNKNOWN',  # unclassifiable beat */
    14: 'NOISE',  # signal quality change */
    16: 'ARFCT',  # isolated QRS-like artifact */
    18: 'STCH',  # ST change */
    19: 'TCH',  # T-wave change */
    20: 'SYSTOLE',  # systole */
    21: 'DIASTOLE',  # diastole */
    22: 'NOTE',  # comment annotation */
    23: 'MEASURE',  # measurement annotation */
    24: 'PWAVE',  # P-wave peak */
    25: 'BBB',  # left or right bundle branch block */
    26: 'PACESP',  # non-conducted pacer spike */
    27: 'TWAVE',  # T-wave peak */
    28: 'RHYTHM',  # rhythm change */
    29: 'UWAVE',  # U-wave peak */
    30: 'LEARN',  # learning */
    31: 'FLWAV',  # ventricular flutter wave */
    32: 'VFON',  # start of ventricular flutter/fibrillation */
    33: 'VFOFF',  # end of ventricular flutter/fibrillation */
    34: 'AESC',  # atrial escape beat */
    35: 'SVESC',  # supraventricular escape beat */
    36: 'LINK',  # link to external data (aux contains URL) */
    37: 'NAPC',  # non-conducted P-wave (blocked APB) */
    38: 'PFUS',  # fusion of paced and normal beat */
    39: 'WFON',  # waveform onset */
    # 39: 'PQ', # PQ junction (beginning of QRS) */
    40: 'WFOFF',  # waveform end */
    # 40: 'JPT', # J point (end of QRS) */
    41: 'RONT'  # R-on-T premature ventricular contraction */
}

In [60]:
import pandas as pd
symcodes = pd.DataFrame({'Ann Symbol': list(annsyms.values()), 'Ann Code/Meaning': list(anncodes.values())})
symcodes = symcodes.set_index('Ann Symbol', list(annsyms.values()))

symcodes

,Ann Code/Meaning
Ann Symbol,
,NOTQRS
N,NORMAL
L,LBBB
R,RBBB
a,ABERR
V,PVC
F,FUSION
J,NPC
A,APC


In [65]:
x = np.array([1,2,3,4,5])
np.where(x == 5)

(array([4]),)

In [71]:
definedcodes = ['N', 'A', 'B', 'C', 'D']


annsymbol = ['N', 'N', 'P,', 'P', 'N', 'G', 'N', 'N', 'C', 'N', 'G']
auxilliarymsg = ['','','','','','','','He is dying','','','He died']


for i in range(0, len(annsymbol)):
    if annsymbol[i] not in definedcodes:
        auxilliarymsg[i] = annsymbol[i]+' '+auxilliarymsg[i]
        annsymbol[i] = ''
        
print(annsymbol)
print(auxilliarymsg)

['N', 'N', '', '', 'N', '', 'N', 'N', 'C', 'N', '']
['', '', 'P, ', 'P ', '', 'G ', '', 'He is dying', '', '', 'G He died']
